In [16]:
import stk
from old_code_test.stk import stk as old_stk
from rdkit.Chem import AllChem as rdkit
import tempfile
from screen_cages import get_properties
import json
from old_code_test.cage_prediction.database.make_database import make_entry

In [69]:
# Load population from old stk 
pop = old_stk.Population.load('/Users/stevenbennett/PhD/main_projects/synthetic_accessibility_project/stages/stage2_screening_workflow/create_image/cage_optimisation/old_code_test/cages/amine2aldehyde3.json', member_init=old_stk.Molecule.from_dict)
fourplussix_gen = (c for c in pop if 'FourPlusSix' in str(c.topology))

In [9]:
def match_cages(old_cage):
    bbs = []
    if not 'FourPlusSix' in str(old_cage.topology):
        print('Different topology error')
        return 
    for bb in old_cage.building_blocks:
        with tempfile.NamedTemporaryFile(mode='w') as temp_f:
            func_group = bb.func_groups[0].info.name
            temp_f.name += '.mol'
            rdk_mol = bb.mol
            rdkit.MolToMolFile(rdk_mol, temp_f.name)
            bbs.append(stk.BuildingBlock.init_from_file(temp_f.name, [func_group]))
    new_cage = stk.ConstructedMolecule(bbs, stk.cage.FourPlusSix())
    with tempfile.NamedTemporaryFile(mode='w') as temp_f:
        temp_f.name += '.mol'
        old_cage.write(temp_f.name)
        new_cage.update_from_file(temp_f.name)
    return (new_cage, old_cage)

In [29]:
new_cage_rdk = new_cage.to_rdkit_mol()

In [40]:
new_cage, old_cage = match_cages(next(fourplussix_gen))

In [41]:
get_properties(new_cage)

{'collapsed': False,
 'window_diff': 0.3589867384048861,
 'max diameter': 33.886213392283906,
 'cavity size': 16.052338298518254}

In [42]:
old_res = make_entry(old_cage)
old_res_d = {
    "collapsed": old_res[4],
    "window_diff": old_res[8],
    "max diameter": old_res[6],
    "cavity size": old_res[5],
}

In [43]:
old_res_d

{'collapsed': False,
 'window_diff': 0.3589867384048861,
 'max diameter': 34.09150311152272,
 'cavity size': 16.069731370552887}

In [44]:
suitable_diffs = {
    "collapsed": 0,
    "window_diff": 0.5,
    "max diameter": 2.0,
    "cavity size": 2.0,
}

In [67]:
def is_close_funcs(old_cage, new_cage):
    old_res = make_entry(old_cage)
    old_res_d = {
        "collapsed": old_res[4],
        "window_diff": old_res[8],
        "max diameter": old_res[6],
        "cavity size": old_res[5],
    }
    new_res = get_properties(new_cage)
    if new_res is None or old_res is None:
        return
    elif None in new_res.values() or None in old_res_d.values():
        return
    for param in old_res_d:
        if abs(old_res_d[param]-new_res[param]) > suitable_diffs[param]:
            print(f'Difference too high for cage {old_cage}.')
            print(f'Parameter is {param}')
            return False
    return True

In [70]:
for i in range(50):
    new_cage, old_cage = match_cages(next(fourplussix_gen))
    print(is_close_funcs(old_cage, new_cage))

True
True
True
True
None
True
None
None
None
None
True
True
True
None
True
True
None
None
None
True
None
None
None
True
None
True
True
True
True
True
True
True
None
None
None
True
None
None
True
True
None
True
None
None
True
None
None
True
True
None
